In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#setup
import warnings
from typing import List, Optional, Union
from datetime import datetime, date
from watttime import api
from watttime.evaluation import report
from watttime.evaluation.get_wt_api_forecast_evaluation_data import AnalysisDataHandler, DataHandlerFactory

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', message="Found version mismatch between `dagster`")

In [3]:
#input parameters
region_list: List[str] = ["ERCOT_SANANTONIO"]
model_date_list: List[str] = ["2022-10-01", "2023-03-01"]
signal_type: str = "co2_moer"
eval_start: Union[str, datetime, date] = '2022-01-01T00:00Z'
eval_end: Union[str, datetime, date] = '2022-12-31T23:59Z'
forecast_sample_days: int = 5

In [4]:
factory = DataHandlerFactory(
    eval_start=eval_start, eval_end=eval_end,
    regions = region_list, model_dates = model_date_list,
    signal_types=signal_type
)

## Data Distribution Changes


Below is a summary of how you can expect signal values to change between versions.

In [5]:
report.plot_sample_moers(factory.data_handlers, max_sample_period='7D')

Warnings Returned: %s | Response: %s {'region': 'ERCOT_SANANTONIO', 'signal_type': 'co2_moer', 'model': '2022-10-01', 'start': datetime.datetime(2022, 1, 1, 0, 0, tzinfo=<UTC>), 'end': datetime.datetime(2022, 1, 30, 23, 55, tzinfo=<UTC>)} {'data_point_period_seconds': 300, 'region': 'ERCOT_SANANTONIO', 'signal_type': 'co2_moer', 'units': 'lbs_co2_per_mwh', 'warnings': [{'type': 'MODEL_MARKED_FOR_DEPRECATION', 'message': "You made a valid request, but the model requested (2022-10-01) is marked for deprecation on 2024-12-04 00:00:00+00:00. After that date, no further data will be created for the requested model. You'll need to specify a newer model from then on, or if you'd like to get data for the newest model, exclude the 'model' parameter from your request."}], 'model': {'date': '2022-10-01', 'type': 'binned_regression'}}
Warnings Returned: %s | Response: %s {'region': 'ERCOT_SANANTONIO', 'signal_type': 'co2_moer', 'model': '2022-10-01', 'start': datetime.datetime(2022, 1, 31, 0, 0, t

In [6]:
report.plot_distribution_moers(factory.data_handlers)

In [9]:
report.plot_region_heatmaps(factory.data_handlers)

## Forecast Performance

We assess the overall performance of our forecasts using a few main metrics, described below.


* **Normalized Mean Absolute Error (Norm MAE):** Mean Absolute Error (MAE) is the average of absolute forecast errors. This represents the magnitude of error to expect from the forecast on average. A model with an MAE closest to 0 is ideal. Norm MAE is MAE divided by the average of the true values. Normalizing MAE allows us to compare models across different datasets. It is expressed as a percentage (%) of the average true value over the validation time period. 
* **Rank Correlation:** A number between -1 and 1 that quantifies the degree to which the rank ordering of two datasets are related. A rank correlation of 1 means the forecast predictions are rank ordered perfectly compared to the true values, making the forecast perfectly effective for time optimization use cases. For many use cases of our forecasts models (such as AER), it is more important that predictions are correctly rank ordered within a forecast window, rather than each forecast being accurate in terms of absolute values.
* **AER CO2 Savings:** This metric represents the impact in lbs CO2/MWh of using the forecast model for energy usage optimization. We compare these CO2 savings against the "Max Potential" case (ie. the case of using perfect forecasts). In this notebook, we model 3 different popular AER Scenarios:
  
  1. EV-night: Charging an electric vehicle for 3 hours out of a 12-hour overnight window starting at 19:00 local time.
  2. EV-day: Charging an electric vehicle for 2 hours out of an 8-hour daytime window starting at 09:00 local time.
  3. Thermostat: Optimizing thermostat usage in continual 1-hour windows throughout a 24-hour period, only actively using energy for half of the time. 



In [13]:
report.plot_norm_mae(factory.data_handlers)

In [14]:
report.plot_rank_corr(factory.data_handlers)

In [26]:
report.plot_impact_forecast_metrics(factory.data_handlers)